<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">R OpenAI Demo</h1>
    </div>
</div>

In [4]:
!conda install -y --quiet -c conda-forge r-irkernel r-rjava r-rjdbc r-remotes

Solving environment: ...working... done

# All requested packages already installed.



<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Switch the kernel from Python to R before running the code below.</p>
    </div>
</div>

In [2]:
library(RJDBC)

Loading required package: DBI

Loading required package: rJava



In [3]:
# URL of the JDBC driver file
driver_url <- "https://repo1.maven.org/maven2/com/singlestore/singlestore-jdbc-client/1.2.1/singlestore-jdbc-client-1.2.1.jar"

# Set the JDBC driver class name
driver <- "com.singlestore.jdbc.Driver"

# Local directory to save the driver file
local_dir <- "jars"
dir.create(local_dir, showWarnings = FALSE, recursive = TRUE)

# Check if the driver file already exists
driver_file <- file.path(
    local_dir,
    "singlestore-jdbc-client-1.2.1.jar"
)

if (!file.exists(driver_file)) {
    # Download the JDBC driver file if it doesn't exist
    download.file(
        driver_url,
        destfile = driver_file,
        mode = "wb",
        quiet = TRUE
    )
}

# Check if the driver file has been downloaded successfully
if (file.exists(driver_file)) {
    print("Driver file downloaded successfully")
} else {
    print("Failed to download the driver file")
}

[1] "Driver file downloaded successfully"


In [24]:
host <- "<HOST>"
port <- 3306
database <- "r_demo"
user <- "admin"
password <- "<PASSWORD>"

url <- paste0("jdbc:singlestore://", host, ":", port, "/", database)

In [5]:
# Establish the JDBC connection
conn <- dbConnect(
    drv = JDBC(driver, driver_file),
    url = url,
    user = user,
    password = password
)

In [6]:
remotes::install_github("ben-aaron188/rgpt3", force = TRUE)

Running `R CMD build`...



* checking for file ‘/tmp/Rtmp7l2lho/remotes1052262b048/ben-aaron188-rgpt3-9fcdd0a/DESCRIPTION’ ... OK
* preparing ‘rgpt3’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* looking to see if a ‘data/datalist’ file should be added
  NB: this package now depends on R (>= 3.5.0)
  serialize/load version 3 cannot be read in older versions of R.
  File(s) containing such objects:
    ‘rgpt3/data/travel_blog_data.RData’
* building ‘rgpt3_1.0.1.tar.gz’


In [7]:
library(jsonlite)
library(rgpt3)

In [22]:
# Prompt for the OpenAI Key
openai_key <- readline(prompt = "OpenAI Key:")

# Specify the file path
file_path <- "access_key.txt"

# Write the string to the file with a newline
writeLines(openai_key, file_path)

OpenAI Key: Your OpenAI Key


In [23]:
rgpt_authenticate("access_key.txt")

[1] "Will use --> Your OpenAI Key for authentication."


In [10]:
data("travel_blog_data")

In [11]:
my_text = travel_blog_data$gpt_content[1]

my_embeddings = rgpt_single_embedding(
    input = my_text,
    model = "text-embedding-3-small"
)

In [12]:
# Create a DataFrame with one row
article <- data.frame(
    id = travel_blog_data$n[1],
    text = my_text,
    vector = as.character(toJSON(my_embeddings))
)

In [13]:
# Write the DataFrame to the database
dbWriteTable(conn, "articles", article, append = TRUE)

In [14]:
# Read the data from the database
article_from_db <- dbReadTable(conn, "articles")

# Print the summary of the DataFrame
str(article_from_db)

'data.frame':	1 obs. of  3 variables:
 $ id    : num 1
 $ text  : chr "Title: Paws Across the UK: A Tail-Wagging Journey Through Britain\n\nAs someone who firmly believes that advent"| __truncated__
 $ vector: chr "[0.0215000007,0.0126999998,0.0524000004,0.0340999998,-0.0328000002,-0.0188999996,-0.0214000009,0.0250000004,-0."| __truncated__


In [15]:
multiple_embeddings = rgpt_embeddings(
    input_var = travel_blog_data$gpt_content,
    id_var = travel_blog_data$n,
    param_model = "text-embedding-3-small"
)

[1] "Embedding: 1/10"
[1] "Embedding: 2/10"
[1] "Embedding: 3/10"
[1] "Embedding: 4/10"
[1] "Embedding: 5/10"
[1] "Embedding: 6/10"
[1] "Embedding: 7/10"
[1] "Embedding: 8/10"
[1] "Embedding: 9/10"
[1] "Embedding: 10/10"


In [16]:
# Convert each row of embeddings to a JSON array
embeddings_json <- apply(multiple_embeddings[, 1:1536], 1, function(row) {
    toJSON((row))
})

# Create the DataFrame with id and vector columns
articles <- data.frame(
    id = multiple_embeddings$id,
    vector = embeddings_json
)

# Merge articles with travel_blog_data based on id and n columns
merged_data <- merge(articles, travel_blog_data[, c("n", "gpt_content")], by.x = "id", by.y = "n", all.x = TRUE)

# Update the articles DataFrame with the merged gpt_content
articles$text <- merged_data$gpt_content

# Reorder the columns to match the database schema
articles <- articles[, c("id", "text", "vector")]

In [17]:
# Write the DataFrame to the database
dbWriteTable(conn, "articles", articles, append = TRUE)

In [18]:
# Read the data from the database
articles_from_db <- dbReadTable(conn, "articles")

# Print the summary of the DataFrame
str(articles_from_db)

'data.frame':	11 obs. of  3 variables:
 $ id    : num  3 5 8 9 7 4 1 6 1 2 ...
 $ text  : chr  "**Paw Prints Across the UK: A Dog's Tail-Wagging Journey**\n\nAs a trusty canine companion with a nose for adve"| __truncated__ "# Paws Across Britain: A Tail-Wagging Journey\n\n## Introduction\n\nHello fellow adventurers and canine compani"| __truncated__ "### A Tail-Wagging Tour: Exploring the UK on Four Paws\n\nThe United Kingdom, a patchwork of historic cities, m"| __truncated__ "### A Dog's Tail: Wagging Through the UK\n\n#### Introduction\n\nHello, fellow furry adventurers and their huma"| __truncated__ ...
 $ vector: chr  "[0.0509000011,0.0228000004,0.0489999987,0.0309999995,-0.0443000011,-0.0141000003,-0.0478000008,0.0162000004,-0."| __truncated__ "[0.0307,-0.00240000011,0.0615000017,0.0234999992,-0.0527999997,-0.00719999988,-0.00400000019,0.0285999998,0.001"| __truncated__ "[0.0226000007,0.0152000003,0.0606000014,0.0329000019,-0.0456000008,-0.0109999999,-0.0384000018,0.0278999992,-0

In [19]:
# Close the JDBC connection
dbDisconnect(conn)

[1] TRUE